# Try a CNN model to check if better results can be achieved

In [2]:
import pandas as pd
import numpy as np


In [3]:
df = pd.read_csv('df_clean.csv')
print(df.shape)
df.head()

(5044, 8)


,Name,Platform,Year,Genre,Publisher,Global_Sales,Meta_Score,User_Score
0,WII SPORTS,WII,2006.0,Sports,Nintendo,82.74,76,8.1
1,MARIO KART WII,WII,2008.0,Racing,Nintendo,35.82,82,8.4
2,WII SPORTS RESORT,WII,2009.0,Sports,Nintendo,33.00,80,8.2
3,NEW SUPER MARIO BROS.,DS,2006.0,Platform,Nintendo,30.01,89,8.5
4,NEW SUPER MARIO BROS. WII,WII,2009.0,Platform,Nintendo,28.62,87,8.3


In [4]:
from keras.models import Sequential
from keras.layers import Dense

Create the bins according to business requirements

In [5]:
df1=df
cut_labels_4 = [0,1,2,3] #['silver', 'gold', 'platinum', 'diamond'],
# cut_bins = [0, 1, 2.5, 5, 100]
df1['target'] = pd.qcut(df1['Global_Sales'], q=4, labels=cut_labels_4)
print(df1.head())
df1['target'].value_counts()

                        Name Platform    Year     Genre Publisher  \
0                 WII SPORTS      WII  2006.0    Sports  Nintendo   
1             MARIO KART WII      WII  2008.0    Racing  Nintendo   
2          WII SPORTS RESORT      WII  2009.0    Sports  Nintendo   
3      NEW SUPER MARIO BROS.       DS  2006.0  Platform  Nintendo   
4  NEW SUPER MARIO BROS. WII      WII  2009.0  Platform  Nintendo   

   Global_Sales  Meta_Score  User_Score target  
0         82.74          76         8.1      3  
1         35.82          82         8.4      3  
2         33.00          80         8.2      3  
3         30.01          89         8.5      3  
4         28.62          87         8.3      3  


0    1304
1    1251
3    1246
2    1243
Name: target, dtype: int64

In [6]:
df1 = pd.get_dummies(data=df, columns=['Platform', 'Genre', 'Publisher'])
print(df1.shape) #post-dummies shape
df1.head() #Check to verify that dummies are ok

(5044, 238)


,Name,Year,Global_Sales,Meta_Score,User_Score,target,Platform_3DS,Platform_DC,Platform_DS,Platform_GBA,...,Publisher_Wargaming.net,Publisher_Warner Bros. Interactive Entertainment,Publisher_White Park Bay Software,Publisher_Xplosiv,Publisher_Xseed Games,Publisher_Yacht Club Games,Publisher_Zoo Digital Publishing,Publisher_Zoo Games,Publisher_Zushi Games,Publisher_bitComposer Games
0,WII SPORTS,2006.0,82.74,76,8.1,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,MARIO KART WII,2008.0,35.82,82,8.4,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,WII SPORTS RESORT,2009.0,33.00,80,8.2,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,NEW SUPER MARIO BROS.,2006.0,30.01,89,8.5,3,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,NEW SUPER MARIO BROS. WII,2009.0,28.62,87,8.3,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
df1 = df1.drop(['Name','Global_Sales'], axis=1)

In [8]:
df1.head()

,Year,Meta_Score,User_Score,target,Platform_3DS,Platform_DC,Platform_DS,Platform_GBA,Platform_GC,Platform_N64,...,Publisher_Wargaming.net,Publisher_Warner Bros. Interactive Entertainment,Publisher_White Park Bay Software,Publisher_Xplosiv,Publisher_Xseed Games,Publisher_Yacht Club Games,Publisher_Zoo Digital Publishing,Publisher_Zoo Games,Publisher_Zushi Games,Publisher_bitComposer Games
0,2006.0,76,8.1,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2008.0,82,8.4,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2009.0,80,8.2,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2006.0,89,8.5,3,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2009.0,87,8.3,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
target=df1['target']
feats=df1.drop(['target'],axis=1)

In [16]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(feats,target,test_size=0.2, random_state=66)
from sklearn import metrics # Pour évaluer les modèles

In [17]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [35]:
# define the keras model
model = Sequential()
model.add(Dense(64, input_dim=235, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(4, activation='softmax'))
# compile the keras model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit the keras model on the dataset
model.fit(X_train, y_train, epochs=20, batch_size=8, validation_split=0.2)

# make class predictions with the model
predictions = model.predict(X_test)
# Evaluation du modèle
score = model.evaluate(X_test, y_test)
print ("score is: ",score)
# confusion matrix
predict_class = np.argmax(predictions, axis=1)
cnf_matrix = metrics.confusion_matrix(y_test, predict_class)
print(cnf_matrix)

Epoch 1/20
404/404 [==============================] - 1s 2ms/step - loss: 1.3387 - accuracy: 0.3386 - val_loss: 1.2915 - val_accuracy: 0.3556
Epoch 2/20
404/404 [==============================] - 1s 1ms/step - loss: 1.1997 - accuracy: 0.4473 - val_loss: 1.1908 - val_accuracy: 0.4659
Epoch 3/20
404/404 [==============================] - 1s 1ms/step - loss: 1.1249 - accuracy: 0.4867 - val_loss: 1.1807 - val_accuracy: 0.4672
Epoch 4/20
404/404 [==============================] - 1s 1ms/step - loss: 1.0723 - accuracy: 0.5198 - val_loss: 1.1673 - val_accuracy: 0.4845
Epoch 5/20
404/404 [==============================] - 1s 1ms/step - loss: 1.0326 - accuracy: 0.5369 - val_loss: 1.1764 - val_accuracy: 0.4845
Epoch 6/20
404/404 [==============================] - 1s 1ms/step - loss: 1.0017 - accuracy: 0.5567 - val_loss: 1.1730 - val_accuracy: 0.4796
Epoch 7/20
404/404 [==============================] - 1s 1ms/step - loss: 0.9735 - accuracy: 0.5623 - val_loss: 1.1874 - val_accuracy: 0.4870
Epoch 